In [1]:
import numpy as np
import pandas as pd

In [2]:
X = pd.read_csv('df_input_to_ga.csv')
target = pd.read_csv('../cleaned-data-frames/target-electricity-consumption.csv')

In [3]:
# making the lengths the same

X = X.iloc[0:7560,:]

In [4]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

X = clean_dataset(X)
target = clean_dataset(target)

In [5]:
X = X.drop(columns = 'Unnamed: 0')

In [6]:
target_vector = target['Consumption']
input = X

In [7]:
# recognising the solution space for this data:

magnitude_of_consumption_values = np.min(target['Consumption'])

magnitude_of_consumption_values

0.0

In [8]:
y = X*10000000

In [9]:
weights_size = len(input.iloc[0])
solutions_per_problem = 100

pop_size = (solutions_per_problem, weights_size)
new_population = np.random.randint(low = 1, high= 50, size = pop_size)

pop_original = new_population

In [10]:
min_values = []
max_values = []

min = 500/y.iloc[13,:]

min_values.append(min)

max = 15000/y.iloc[13,:]
max_values.append(max)

In [11]:
min_values

[AWT26_Power(albany)                  0.853449
 AWP_200_Power(albany)                0.811716
 D2CF_200_Power(albany)               0.500952
 AN_Bonus_150/30_Power(albany)        0.510582
 Aria_20_Power(albany)                7.385547
 WWD-1_D64_Power(albany)              0.127686
 Solar_AllMax(albany)                 0.370261
 Solar_TrinaSolar(albany)             0.370261
 Solar_Mitsubishi(albany)             0.371088
 Solar_FirstSolar(albany)             0.370747
 AWT26_Power(buffalo)                 0.492626
 AWP_200_Power(buffalo)               0.585086
 D2CF_200_Power(buffalo)              0.344386
 AN_Bonus_150/30_Power(buffalo)       0.381282
 Aria_20_Power(buffalo)               5.456540
 WWD-1_D64_Power(buffalo)             0.089996
 AWT26_Power(binghamton)              0.853449
 AWP_200_Power(binghamton)            0.811716
 D2CF_200_Power(binghamton)           0.500952
 AN_Bonus_150/30_Power(binghamton)    0.510582
 Aria_20_Power(binghamton)            7.385547
 WWD-1_D64_Po

In [12]:
max_values

[AWT26_Power(albany)                   25.603483
 AWP_200_Power(albany)                 24.351488
 D2CF_200_Power(albany)                15.028549
 AN_Bonus_150/30_Power(albany)         15.317453
 Aria_20_Power(albany)                221.566406
 WWD-1_D64_Power(albany)                3.830584
 Solar_AllMax(albany)                  11.107825
 Solar_TrinaSolar(albany)              11.107825
 Solar_Mitsubishi(albany)              11.132629
 Solar_FirstSolar(albany)              11.122402
 AWT26_Power(buffalo)                  14.778787
 AWP_200_Power(buffalo)                17.552568
 D2CF_200_Power(buffalo)               10.331582
 AN_Bonus_150/30_Power(buffalo)        11.438469
 Aria_20_Power(buffalo)               163.696205
 WWD-1_D64_Power(buffalo)               2.699871
 AWT26_Power(binghamton)               25.603483
 AWP_200_Power(binghamton)             24.351488
 D2CF_200_Power(binghamton)            15.028549
 AN_Bonus_150/30_Power(binghamton)     15.317453
 Aria_20_Power(bingh